In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings(action = "ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/train.csv")
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/test.csv")

In [ ]:
print(test_df)

In [ ]:
print(len(train_df))
summary = pd.DataFrame(train_df.dtypes)
summary["Null"] = train_df.isnull().sum()
summary["first"] = train_df.loc[0]
summary["second"] = train_df.loc[1]
summary["third"] = train_df.loc[2]
summary["unique"] = train_df.nunique()
summary.tail(10)

We have 74 features, every feature is non null and integer so no imputations required. 200,000 rows are there

# Univariate Analysis

In [ ]:
def get_row_col_idx(idx):
    """Gets the index of row and col to plot"""
    row_idx = idx//6
    col_idx = idx%6
    return row_idx, col_idx

In [ ]:
fig, ax = plt.subplots(13,6,figsize = (30,30))
for idx, cols in enumerate(train_df.columns.to_list()[1:-1]):
    row_idx, col_idx = get_row_col_idx(idx)
    sns.distplot(train_df[cols], ax = ax[row_idx,col_idx], color = "Blue")
    sns.distplot(test_df[cols], ax = ax[row_idx,col_idx], color = "yellow")

We can see from here that the data is skewed towards right.

In [ ]:
def write_percent(ax, train_df):
    """Writes the percentage on top of bars in barplot."""
    for patch in ax.patches:
        height = patch.get_height()
        width = patch.get_width()
        x_loc = patch.get_x()
        percent = height/len(train_df)*100
        ax.text(x = x_loc + width/2, y = height, s = '{:.1f}'.format(percent))


In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
sns.histplot(train_df["target"], color = "Red", ax = ax)
write_percent(ax, train_df)

From here we could see that Class_6 and Class_8 are frequent class ann on the other hand Class_4 and Class_5 are the classes which have occured quite a few times. There are 9 unique values for target variable.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

corr = train_df.corr()
f, ax = plt.subplots(figsize=(30,30))
sns.heatmap(corr, vmax=.8, square=True);


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = train_df.pop("target")
y = encoder.fit_transform(y)
print(pd.Series(y).value_counts())

In [ ]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, precision_score, recall_score
id_col = train_df.pop("id")
X = train_df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1, stratify=y)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1, stratify = y) # 0.25 x 0.8 = 0.2

In [ ]:
# model = RandomForestClassifier(n_estimators=300, max_depth=15, n_jobs=-1)
model = XGBClassifier(learning_rate =0.1,max_depth=5, n_estimators=200)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_train)
print(y_pred)

In [ ]:
def log_loss(pred_probs, y_valid):
    pred_probs = pred_probs + 1e-15 # numerical stability
    y_valid = y_valid.reshape(-1, 1) 
    
    # get probs for each class
    preds_class = np.take_along_axis(pred_probs, y_valid, axis=1)
    return -1 * np.log(preds_class).mean()

In [ ]:
preds = model.predict_proba(X_train)
log_loss(preds, y_train)

In [ ]:
preds = model.predict_proba(X_test)
log_loss(preds, y_test)

In [ ]:
# model = XGBClassifier(learning_rate =0.1,max_depth=5, n_estimators=200)
# model.fit(X_train, y_train)
# y_pred = model.predict_proba(X_test)
# score = log_loss(y_pred, y_test)

In [ ]:
# print(score)

In [ ]:
test_df = test_df.iloc[:, 1:]

In [ ]:

test_preds = model.predict_proba(test_df)
sub = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv")
sub.iloc[:, 1:] = test_preds
sub.to_csv("submission.csv", index=False)